In [159]:
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.filters import gaussian_filter1d
import numpy as np

normalise_x = []
normalise_y = []

# global variables
bg = None
canvas = np.ones([720,1080 , 3] , np.uint8)
first_time  =  False
# global old_x , old_y
# old_x = 0
# old_y = 0


x = np.array([])
y = np.array([])

#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

    
    
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    thresholded = cv2.morphologyEx(thresholded , cv2.MORPH_CLOSE , (21,21))
    thresholded = cv2.dilate(thresholded , (21,21) , iterations = 5)
    
    # get the contours in the thresholded image
    cnts,_ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)
    
    

    
    
    
    
def smoothen_curve():
    global normalise_x,normalise_y
    
    normalise_x = gaussian_filter1d(normalise_x, sigma=2)
    normalise_y = gaussian_filter1d(normalise_y, sigma=2)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

def findPoints(f , contours):
    peri = cv2.arcLength(contours , True)
    approx = cv2.approxPolyDP(contours , 0.05 * peri , True)
    if cv2.contourArea(max(contours,key = cv2.contourArea)) > 100:
        f = cv2.drawContours(f , [approx] , -1 , (255,0,0) , 5)
    

    min = 1000000
    max_x = 0
    
    for i in range(len(approx)):
        if(min>approx[i][0][1]):
            min = approx[i][0][1]
            max_x = approx[i][0][0]
            
    global x , y 
    
    x = np.append(x , [int(max_x)])
    y = np.append(y , [int(min)])
    
#     if old_x == 0 and old_y == 0:
#         old_x , old_y = int(max_x) , int(min)
        
#     old_x , old_y = int(max_x) , int(min)
    
    f = cv2.circle(f , (int(max_x),int(min)) , 5 , (0,0,0) , 10)

#     old_x = max_x
#     old_y = min
    
    
    return f
    
    
    
    
    
    
    
    
    
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    vid = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 300, 360, 900

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        success, frame = vid.read()

        # resize the frame
        frame = cv2.resize(frame, (1080,720))

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()
        

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thresholded", thresholded)
                clone = findPoints(clone , segmented + (right, top))
        
            else:
                cv2.destroyWindow("Thresholded")
                
        




        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames = 1 + num_frames
        
        
        
        
        try:
            
            old_x = x[len(x)-1]
            old_y = y[len(y)-1]

            if(cv2.waitKey(33) == ord("a")):
                
#                 cv2.circle(canvas , (int(old_x),int(old_y)), 2 , (255,0,0) , 4)
                
#                 cv2.line(canvas , (int(old_x),int(old_y)), (int(x[len(x)-1]),int(y[len(y)-1])) , (255,255,255) , 4)
                

    

                normalise_x.append(int(old_x))
                normalise_y.append(int(old_y))
            
                if (first_time == True):
                    first_time = False
                    continue
                
                

                for i in range(len(normalise_x) - 1 , len(normalise_x)):
                    x_old = int(normalise_x[i-1])
                    y_old = int(normalise_y[i-1])


                    x_new = int(normalise_x[i])
                    y_new = int(normalise_y[i])
                    
                    
                    cv2.line(canvas , (x_old , y_old) , (x_new , y_new) , (255,255,255) , 4)
            else:
                first_time = True



            
        except:
            pass
        

        old_x = 0
        old_y = 0
        cv2.imshow("Canvas",cv2.add(canvas ,clone))
        
        
        
        
        

#         # display the frame with segmented hand
#         cv2.imshow("Video Feed",clone)

        
        if cv2.waitKey(33) == 27:
            break

# free up memory
vid.release()
cv2.destroyAllWindows()



# smoothen_curve()



In [160]:


# normalise_x = np.array(normalise_x)
# normalise_y = np.array(normalise_y)

In [161]:
from scipy.ndimage.filters import gaussian_filter1d
normalise_x = gaussian_filter1d(normalise_x, sigma=1)
normalise_y = gaussian_filter1d(normalise_y, sigma=1)

In [162]:
canvas = np.ones([300,640] , np.uint8)
y = np.flip(normalise_y, axis =0)
TOP = 0
LEFT = 300

for i in range(1 , len(normalise_x)):
    x_old = int(normalise_x[i-1]) - LEFT
    y_old = int(normalise_y[i-1]) - TOP
    
    x_new = int(normalise_x[i]) - LEFT
    y_new = int(normalise_y[i]) - TOP
    cv2.line(canvas , (x_old , y_old) , (x_new , y_new) , (255,255,255) , 4)



# for i in range (1,len(normalise_x)):
#     cv2.line(canvas ,  (int(normalise_x[len(normalise_x)-2]),int(normalise_y[len(normalise_y)-2])), (int(normalise_x[len(normalise_x)-1]),int(normalise_y[len(normalise_y)-1080])) , (255,255,255) , 4)


cv2.imshow("Canvas",canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [129]:
cv2.imwrite(r"C:\Users\user\Desktop\signature.jpg", canvas)

True